In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
import importlib
import common_utils
importlib.reload(common_utils)
import pandas as pd
import itertools

import tensorflow as tf
from common_utils import get_unique_image_shapes,get_unique_image_paths,load_images_from_paths,build_image_dataframe,split_data, bin_ages, build_cnn_model,build_model_from_config

In [2]:


# tf.keras.mixed_precision.set_global_policy('mixed_float16')

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
image_paths_csv = pd.read_csv('./processed_data/image_paths.csv')

In [4]:
paths_train_df, paths_val_df, paths_test_df = split_data(image_paths_csv)

In [5]:
configurations = {
    'channels': [3],
    'use_skip': [True, False],
    'num_conv_layers': [3, 4],
    'base_filters': [16,32, 64,128,256],
    'kernel_size': [3, 5,7],
    'activation': ['relu', 'elu','leakyrelu','tanh','swish'],
    'num_dense_layers': [1, 2,3,4,5],
    'dense_units': [32,64,128],
    'dropout_rate': [0.3, 0.5,0.7],
    'output_activation': ['sigmoid', 'softmax'],
}


In [6]:


all_combinations = list(itertools.product(*configurations.values()))
valid_configs = []

for combo in all_combinations:
    params = dict(zip(configurations.keys(), combo))
    
    # Enforce: If use_skip=True, use_pooling=False
    if params['use_skip']:
        params['use_pooling'] = False
    else:
        params['use_pooling'] = True  # Or add to search space
    
    valid_configs.append(params)



In [7]:
train_dataset = load_images_from_paths(paths_train_df, channels=3)
val_dataset = load_images_from_paths(paths_val_df, channels=3)


2025-04-01 13:23:52.675340: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-04-01 13:23:52.675369: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-01 13:23:52.675380: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-01 13:23:52.675398: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-01 13:23:52.675407: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
def configure_dataset(dataset, shuffle=False):
    if shuffle:
        dataset = dataset.shuffle(buffer_size=500)  # Adjust buffer size as needed
    dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    return dataset


In [9]:
train_dataset = configure_dataset(train_dataset, shuffle=True)
val_dataset = configure_dataset(val_dataset)


In [ ]:
model_dict = {}
model_dir = "saved_models"
os.makedirs(model_dir, exist_ok=True)
paths_pre_train_model_dict = {}


for i, config in enumerate(valid_configs):
    # Generate unique model name
    name_parts = [
        f"{config['channels']}ch",
        f"skip_{config['use_skip']}",
        f"conv{config['num_conv_layers']}",
        f"k{config['kernel_size']}",
        config['activation'],
        f"dense{config['num_dense_layers']}x{config['dense_units']}",
        f"drop{config['dropout_rate']}",
        f"out_{config['output_activation']}"
    ]
    
    model_name = "_".join(name_parts)
    model_filename = model_name + ".keras"
    model_path = os.path.join(model_dir, model_filename)

    if os.path.exists(model_path):
        print(f"Skipped {model_name} (already exists)")
        paths_pre_train_model_dict[model_name] = {
            'model_path': model_path,
            'config': config
        }
        continue
    # Build and compile
    model = build_model_from_config(config)
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',  # Fixed loss function
        metrics=['accuracy']
    )
    model.save(model_path,overwrite=True)
    
    paths_pre_train_model_dict[model_name] = {
        'model_path': model_path,
        'config': config
    }

    # Train
    # history = model.fit(
    #     train_dataset,  # Dataset yields (images, targets)
    #     validation_data=val_dataset,
    #     epochs=50,
    #     batch_size=32,
    #     verbose=0,
    #     callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)]
    # )
    
    # Store results
    # model_dict[model_name] = {
    #     'model': model,
    #     'history': history.history,
    #     'config': config
    # }
    # print(f"Trained {model_name} | Val acc: {max(history.history['val_accuracy']):.4f}")
    print(f"Created {model_name} ")


Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.3_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.3_out_softmax (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.5_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.5_out_softmax (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.7_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x128_drop0.7_out_softmax (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.3_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.3_out_softmax (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.5_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.5_out_softmax (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.7_out_sigmoid (already exists)
Skipped 3ch_skip_True_conv3_k3_relu_dense1x256_drop0.7_out_softmax (already 